# PA4 Simple Classifiers (Naive Bayes)
## Part 2: Auto Dataset Classification  
Write a Jupyter Notebook (pa4.ipynb) that uses your mysklearn package to build simple classifiers for the "pre-processed" automobile dataset (auto-data-removed-NA.txt) you created for PA2. In the Notebook, describe the steps, log any assumptions and/or issues you had in doing the steps, and provide insights on the step results. All re-usable utility functions should be separate from your Notebook in an appropriate module.

In [14]:
# some useful mysklearn package import statements and reloads
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myutils as myutils
importlib.reload(myutils)

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MySimpleLinearRegressor, MyNaiveBayesClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

### Step 2 Train/Test Sets: Random Instances and kNN  
Create a nearest neighbor classifier for mpg that uses the number of cylinders, weight, and acceleration attributes to predict mpg for k = 5. Be sure to normalize the three attribute values and also use the Euclidean distance metric. Similar to Step 1, test your classifier by selecting 5 random instances from the dataset, predict their corresponding mpg ranking, and then show their actual mpg ranking:
* Changes:
    * A scale function needed to be made to scale the data to be between [0, 1]

In [15]:
import os
importlib.reload(myutils)

# Get the file data
fname = os.path.join("input_data", "auto-data-removed-NA.csv")
auto_data = MyPyTable().load_from_file(fname)
auto_data.remove_rows_with_missing_values()

# Grab the car cylinders, weight, model year and store in a list
cylinders = auto_data.get_column('cylinders')
weight = myutils.convert_weight(auto_data.get_column('weight'))
model_year = auto_data.get_column('model year')

# split the data
X_train = [[cylinders[i],weight[i],model_year[i]] for i in range(len(cylinders))]
y_train = myutils.convert_rating(auto_data.get_column('mpg'))

# Fit to the Naive Bayes Classifier
mnbc = MyNaiveBayesClassifier()
mnbc.fit(X_train, y_train)

# randomize the data for 5 rows
rand_rows = myutils.get_rand_rows(auto_data, 5)
print("========================================")
print("STEP 2: Naive Bayes 5 random predictions")
print("========================================")

# for row in the random row list
X_test = [[row[1],myutils.get_weight(row[4]),row[6]] for row in rand_rows]
actual = [myutils.get_rating(row[0]) for row in rand_rows]

# predict the values
predicted = mnbc.predict(X_test)
myutils.prediction_pretty_print(rand_rows,actual,predicted)

STEP 2: Naive Bayes 5 random predictions
instance: [22.0, 6.0, 198.0, 95.0, 2833.0, 15.5, 70.0, 1.0, 'plymouth duster', 2547.0]
class: 5 actual: 5
instance: [14.0, 8.0, 318.0, 150.0, 4457.0, 13.5, 74.0, 1.0, 'dodge coronet custom (sw)', 3882.0]
class: 2 actual: 2
instance: [26.0, 4.0, 91.0, 70.0, 1955.0, 20.5, 71.0, 1.0, 'plymouth cricket', 1915.0]
class: 7 actual: 6
instance: [27.2, 4.0, 141.0, 71.0, 3190.0, 24.8, 79.0, 2.0, 'peugeot 504', 8040.0]
class: 5 actual: 7
instance: [26.5, 4.0, 140.0, 72.0, 2565.0, 13.6, 76.0, 1.0, 'ford pinto', 3025.0]
class: 5 actual: 6


### Step 3 Train/Test Sets: Random Sub-sampling
Compute the predictive accuracy and error rate of the two classifiers using random sub-sampling with k = 10. Your output should look something like this (where the ??'s should be replaced by actual values):
* This one was a bit difficult but didnt require me to create any new funcitons to pull it off.
* Data:
    * The accuracy seemed to run a bit low which means that the algorithm was not regularly correct in its predictions. This also means the error rate was high.

In [16]:
importlib.reload(myutils)

print("========================================")
print("STEP 3: Naive Bayes predictive accuracy")
print("========================================")
print('Random Subsample (2:1 Train/Test)')

# split the data
split_xtrain, split_xtest, split_ytrain, split_ytest = myevaluation.train_test_split(X_train, y_train, shuffle=True)

# Fit the data for Naive Bayes
mnbc.fit(split_xtrain,split_ytrain)

# Predict using Naive Bayes
predicted = mnbc.predict(split_xtest)

# Print the accuracy data
accuracy = myutils.get_accuracy(split_ytest, predicted)
print('Naive Bayes: accuracy =', accuracy, 'error rate = ', (1-accuracy))

STEP 3: Naive Bayes predictive accuracy
Random Subsample (2:1 Train/Test)
Naive Bayes: accuracy = 0.38372093023255816 error rate =  0.6162790697674418


### Step 4 Train/Test Sets: Cross Validation
Compute the predictive accuracy and error rate of the two classifiers using separate training and test sets. You should use mpg rankings for both k-fold cross validation and stratified k-fold cross validation with k = 10. Your output should look something like this (where the ??'s should be replaced by actual values):
* Corrections:
    * I needed to make a function that counted the predictions that were correct. 
* Data:
    * The predictions for the cross validation were actually pretty high menaing it was able to predict rather well.
    * However the stratified k fold was not regularly very accurate.

In [17]:
print("===========================================")
print("STEP 4: Naive Bayes Predictive Accuracy")
print("===========================================")
print("10-Fold Cross Validation")

# fold the column data
train_folds, test_folds = myevaluation.kfold_cross_validation(X_train, 10)
fold_xtrain, fold_ytrain, fold_xtest, fold_ytest = myutils.get_from_folds(X_train, y_train, train_folds, test_folds)

# Fit the data using Naive Bayes
mnbc.fit(fold_xtrain, fold_ytrain)

# Predict the data using Naive Bayes
predicted = mnbc.predict(fold_xtest)
accuracy = myutils.get_accuracy(fold_ytest, predicted)
print('Naive Bayes: accuracy =', accuracy, 'error rate = ', (1-accuracy))

print()
print("Stratified 10-Fold Cross Validation")

# Make the folds
strattrain_folds, strattest_folds = myevaluation.stratified_kfold_cross_validation(X_train, y_train, 10)

# get the train and test lists
strat_xtrain, strat_ytrain, strat_xtest, strat_ytest = myutils.get_from_folds(X_train, y_train, train_folds, test_folds)

# Fit the data using Naive Bayes
mnbc.fit(strat_xtrain, strat_ytrain)

# Predict the data using Naive Bayes
predicted = mnbc.predict(strat_xtest)
accuracy = myutils.get_accuracy(strat_ytest, predicted)
print('Naive Bayes: accuracy =', accuracy, 'error rate = ', (1-accuracy))

STEP 4: Naive Bayes Predictive Accuracy
10-Fold Cross Validation
Naive Bayes: accuracy = 0.5366795366795367 error rate =  0.4633204633204633

Stratified 10-Fold Cross Validation
Naive Bayes: accuracy = 0.5366795366795367 error rate =  0.4633204633204633


### Step 5 Confusion Matrices  
Create confusion matrices for each classifier based on the stratified 10-fold cross validation results. You can use the tabulate package to display your confusion matrices (it is also okay to format the table manually). Here is an example:
* Data:
    * It seems the matrix was not able to guess the correct label very often.

In [13]:
importlib.reload(myutils)

print("===========================================")
print("STEP 5: Confusion Matrices")
print("===========================================")

# create the confusion matrix
matrix = myevaluation.confusion_matrix(strat_ytest, predicted, [0,1,2,3,4,5,6,7,8,9,10])
table_header = ['MPG', 1,2, 3 ,4, 5, 6, 7, 8, 9, 10, 'Total', 'Recognition (%)']

# Prepare the table to be printed
myutils.add_conf_stats(matrix)
myutils.print_tabulate(matrix, table_header)


STEP 5: Confusion Matrices

=====  ===  ===  ===  ===  ===  ===  ===  ===  ===  ====  =======  =================
  MPG    1    2    3    4    5    6    7    8    9    10    Total    Recognition (%)
=====  ===  ===  ===  ===  ===  ===  ===  ===  ===  ====  =======  =================
    1   26    1    1    0    0    0    0    0    0     0       29              89.66
    2    8    6    2    0    0    0    0    0    0     0       18              33.33
    3    8    6   13    9    0    0    0    0    0     0       39              33.33
    4    2    1   10   31    5    2    0    0    0     0       55              56.36
    5    0    0    2   16   20    9    0    0    0     0       52              38.46
    6    0    0    0    1    6   22    4    1    0     0       40              55
    7    0    0    0    0    4    9   11    3    0     0       34              32.35
    8    0    0    0    0    1    1    6   10    0     0       26              38.46
    9    0    0    0    0    0    0    